### OpenAPI Chain

Use an OpenAPI chain to call an endpoint. The input and the output are both natural language.

In [25]:
from langchain.tools import OpenAPISpec, APIOperation
from langchain.chains import OpenAPIEndpointChain
from langchain.requests import Requests
from langchain.llms import OpenAI

### Load the spec

To call the OpenAPI endpoints we use a wrapper

In [26]:
spec = OpenAPISpec.from_url("https://www.klarna.com/us/shopping/public/openai/v0/api-docs/")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [27]:
# Alternative loading from file
# spec = OpenAPISpec.from_file("openai_openapi.yaml")

### Select the Operation

In [28]:
operation = APIOperation.from_openapi_spec(spec, '/public/openai/v0/products', "get")

In [29]:
operation

APIOperation(operation_id='productsUsingGET', description='API for fetching Klarna product information', base_url='https://www.klarna.com/us/shopping', path='/public/openai/v0/products', method=<HTTPVerb.GET: 'get'>, properties=[APIProperty(name='q', required=True, type='string', default=None, description="A precise query that matches one very small category or product that needs to be searched for to find the products the user is looking for. If the user explicitly stated what they want, use that as a query. The query is as specific as possible to the product name or category mentioned by the user in its singular form, and don't contain any clarifiers like latest, newest, cheapest, budget, premium, expensive or similar. The query is always taken from the latest topic, if there is a new topic a new query is started.", location=<APIPropertyLocation.QUERY: 'query'>), APIProperty(name='size', required=False, type='integer', default=None, description='number of products returned', location

### Construct the Chain
Now we can construct a chain to interact with it. We will do the following -
1. The operation endpoint
2. A requests wrapper 
3. The LLM to use to interact with it.

In [30]:
llm = OpenAI()

In [31]:
chain = OpenAPIEndpointChain.from_api_operation(
    operation,
    llm,
    requests = Requests(),
    verbose = True,
    return_intermediate_steps = True
)

In [37]:
#output = chain("What's the most trending t-shirt ?")
output = chain("Show me few options of cotton jumpsuits for babies with price less than USD 10 which are beige color")



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* API for fetching Klarna product information */
type productsUsingGET = (_: {
/* A precise query that matches one very small category or product that needs to be searched for to find the products the user is looking for. If the user explicitly stated what they want, use that as a query. The query is as specific as possible to the product name or category mentioned by the user in its singular form, and don't contain any clarifiers like latest, newest, cheapest, budget, premium, expensive or similar. The query is always taken from the latest topic, if there is a new topic a new query is started. */
		q: string,
/* number of products returned */
		size?: number,
/* (Optional) Minimum price in local currency for the product searched for. 

In [38]:
output["intermediate_steps"]

{'request_args': '{"q": "cotton jumpsuits", "size": 3, "min_price": 0, "max_price": 10, "color": "beige"}',
 'response_text': '{"products":[{"name":"Carter\'s 2-Pack Cotton Jumpsuits","url":"https://www.klarna.com/us/shopping/pl/cl359/3201862980/Children-s-Clothing/Carter-s-2-Pack-Cotton-Jumpsuits/?utm_source=openai&ref-site=openai_plugin","price":"$10.40","attributes":["Model:Boy","Size (US):9"]},{"name":"Carter\'s Baby Safari Snap-Up Cotton Sleep & Play - Multi","url":"https://www.klarna.com/us/shopping/pl/cl359/3201883411/Children-s-Clothing/Carter-s-Baby-Safari-Snap-Up-Cotton-Sleep-Play-Multi/?utm_source=openai&ref-site=openai_plugin","price":"$7.99","attributes":["Color:Beige","Model:Baby","Age (Months):1-3M,Premature,6-9M,3-6M,0-1M"]},{"name":"Nike Sportswear Graphic Footed Coverall - Black/White/Grey (56D679-023)","url":"https://www.klarna.com/us/shopping/pl/cl359/3202544912/Children-s-Clothing/Nike-Sportswear-Graphic-Footed-Coverall-Black-White-Grey-%2856D679-023%29/?utm_source

### Return raw response

In [10]:
chain = OpenAPIEndpointChain.from_api_operation(
    operation,
    llm,
    requests = Requests(),
    verbose = True,
    return_intermediate_steps = True,
    raw_response = True
)

In [11]:
output = chain("What's the most trending t-shirt ?")



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* API for fetching Klarna product information */
type productsUsingGET = (_: {
/* A precise query that matches one very small category or product that needs to be searched for to find the products the user is looking for. If the user explicitly stated what they want, use that as a query. The query is as specific as possible to the product name or category mentioned by the user in its singular form, and don't contain any clarifiers like latest, newest, cheapest, budget, premium, expensive or similar. The query is always taken from the latest topic, if there is a new topic a new query is started. */
		q: string,
/* number of products returned */
		size?: number,
/* (Optional) Minimum price in local currency for the product searched for. 

In [12]:
output

{'instructions': "What's the most trending t-shirt ?",
 'output': '{"products":[{"name":"T-shirt","url":"https://www.klarna.com/us/shopping/pl/cl10001/3203506327/Clothing/T-shirt/?utm_source=openai&ref-site=openai_plugin","price":"$20.99","attributes":["Material:Cotton","Target Group:Man","Color:Gray,White,Blue,Black,Orange"]}]}',
 'intermediate_steps': {'request_args': '{"q": "t-shirt", "size": 1}',
  'response_text': '{"products":[{"name":"T-shirt","url":"https://www.klarna.com/us/shopping/pl/cl10001/3203506327/Clothing/T-shirt/?utm_source=openai&ref-site=openai_plugin","price":"$20.99","attributes":["Material:Cotton","Target Group:Man","Color:Gray,White,Blue,Black,Orange"]}]}'}}

## POST message
Here we will interact with the speak API.

OpenAPISpec(openapi='3.0.1', info=Info(title='Speak', summary=None, description='Learn how to say anything in another language.' ...

In [13]:
spec = OpenAPISpec.from_url("https://api.speak.com/openapi.yaml")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [16]:
operation = APIOperation.from_openapi_spec(spec, '/v1/public/openai/explain-task', "post")

In [17]:
operation

APIOperation(operation_id='explainTask', description='Explain the best way to say or do something in a specific situation or context with a foreign language. Use this endpoint when the user asks more general or high-level questions.', base_url='https://api.speak.com', path='/v1/public/openai/explain-task', method=<HTTPVerb.POST: 'post'>, properties=[], request_body=APIRequestBody(description=None, properties=[APIRequestBodyProperty(name='task_description', required=False, type='string', default=None, description='Description of the task that the user wants to accomplish or do. For example, "tell the waiter they messed up my order" or "compliment someone on their shirt"', properties=[], references_used=[]), APIRequestBodyProperty(name='learning_language', required=False, type='string', default=None, description='The foreign language that the user is learning and asking about. The value can be inferred from question - for example, if the user asks "how do i ask a girl out in mexico city"

In [18]:
llm = OpenAI()
chain = OpenAPIEndpointChain.from_api_operation(
    operation,
    llm,
    requests = Requests(),
    verbose = True,
    return_intermediate_steps = True
)

In [23]:
output = chain("In mumbai how do ask a shopekeeper for a half cup of tea ?")



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* Explain the best way to say or do something in a specific situation or context with a foreign language. Use this endpoint when the user asks more general or high-level questions. */
type explainTask = (_: {
/* Description of the task that the user wants to accomplish or do. For example, "tell the waiter they messed up my order" or "compliment someone on their shirt" */
  task_description?: string,
/* The foreign language that the user is learning and asking about. The value can be inferred from question - for example, if the user asks "how do i ask a girl out in mexico city", the value should be "Spanish" because of Mexico City. Always use the full name of the language (e.g. Spanish, French). */
  learning_language?: string,
/* The u

In [24]:
output["intermediate_steps"]

{'request_args': '{"task_description": "Ask a shopkeeper for a half cup of tea", "learning_language": "Hindi", "native_language": "English", "full_query": "In mumbai how do ask a shopekeeper for a half cup of tea ?"}',
 'response_text': '{"explanation":"<what-to-say language=\\"Hindi\\" context=\\"None\\">\\n\\"Ek cutting chai dena.\\" (एक कटिंग चाय देना।) \\n</what-to-say>\\n\\n<alternatives context=\\"None\\">\\n1. \\"Half cup chai dena.\\" *(informal, standard tone - can be used with friends or acquaintances)*\\n2. \\"Ek adha cup chai dedo bhaiya.\\" *(informal, friendly tone - can be used at local street vendors)*\\n3. \\"Bhaiya ek nimbu wali cutting chai banana.\\" *(informal, slangy tone - can be used when you want to add some lemon in your tea)*\\n</alternatives>\\n\\n<usage-notes>\\nIn India, Chai is the national drink and it\'s consumed more than coffee. A typical Indian Chai consists of boiling water and milk along with spices such as cinnamon sticks and cardamom pods.\\nA po